In [3]:
import torch 
from torchvision.models.feature_extraction import (
    create_feature_extractor,
    get_graph_node_names,
)
import sys
sys.path.append("/Users/tizianocausin/Desktop/backUp20240609/summer2025/ponce_lab/exp_set/python_scripts/src/")
from alignment.utils import get_maxpool_evecs
from dim_redu_anns.utils import get_layer_out_shape
from torchvision import transforms, datasets, models
import joblib
import numpy as np
from sklearn.cross_decomposition import CCA

In [4]:
model_name = "alexnet"
layer_names = ['features.4', 'features.7']
device = "cpu"
model_cls = getattr(models, model_name)
model = model_cls(pretrained=True).to(device).eval()
feature_extractors = [create_feature_extractor(
    model, return_nodes=[l_n]
).to(device) for l_n in layer_names]


/Users/tizianocausin/Desktop/virtual_envs/ponce_env/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/tizianocausin/Desktop/virtual_envs/ponce_env/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [5]:
layer_shapes = [get_layer_out_shape(feature_extractors[i], layer_names[i]) for i in range(2)]

In [6]:
path2files = [f"/Users/tizianocausin/Library/CloudStorage/OneDrive-SISSA/data_repo/exp_set_res/silico/imagenet_val_{model_name}_{l_n}_pca_model_1000_PCs.pkl" for l_n in layer_names]
data = [joblib.load(p2f) for p2f in path2files]

In [7]:
evecs = [d.components_ for d in data]

In [8]:
maxpool_evecs = [get_maxpool_evecs(data[i], layer_names[i], layer_shapes[i]) for i in range(2)]
del evecs

In [30]:
# TODO work with CCA now
cca = CCA(n_components = 20)
cca.fit(d1,d2)
d1_cca, d2_cca = cca2.fit_transform(maxpool_evecs[0], maxpool_evecs[1])

/Users/tizianocausin/Desktop/virtual_envs/ponce_env/lib/python3.12/site-packages/sklearn/cross_decomposition/_pls.py:104: ConvergenceWarning: Maximum number of iterations reached
  warnings.warn("Maximum number of iterations reached", ConvergenceWarning)


In [31]:
a = np.corrcoef(d1_cca.T, d2_cca.T)#[0,1]

In [32]:
a

np.float64(-5.382206809626199e-16)

In [20]:
a.shape

(10, 10)